In [ ]:
import xarray as xr
import numpy as np
import rioxarray
import h5netcdf
import h5py
import netCDF4
import scipy.ndimage

import hvplot.xarray
import geoviews as gv

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
surface_velocity = xr.open_dataset('data/GRE_G0120_0000.nc')
surface_velocity_spacing_m = np.abs(np.median(np.diff(surface_velocity['x'])))
assert(np.abs(np.max(np.diff(surface_velocity['x']))) == surface_velocity_spacing_m)
assert(np.abs(np.min(np.diff(surface_velocity['x']))) == surface_velocity_spacing_m)
assert(np.abs(np.max(np.diff(surface_velocity['y']))) == surface_velocity_spacing_m) # TODO fix these
assert(np.abs(np.min(np.diff(surface_velocity['y']))) == surface_velocity_spacing_m)

age_model = xr.open_dataset('data/RRRAG4_Greenland_1993_2013_01_age_grid.nc',
                               engine='h5netcdf').rename(
                                   {'number of grid points in x-direction': 'x', 'number of grid points in y-direction': 'y', 'number of vertical layers': 'layers', 'number of isochrones': 'original_isochrones',
                                    'x': 'x_km', 'y': 'y_km'}
                                   )
# Convert km to meters for consistency
age_model = age_model.assign_coords({'x':age_model.x_km[:,0]*1000, 'y':age_model.y_km[0,:]*1000})

bedmachine = xr.open_dataset('data/BedMachineGreenland-v5.nc')
h = bedmachine['thickness'].interp(x=surface_velocity['x'], y=surface_velocity['y'])

bedmachine_spacing_m = np.abs(np.median(np.diff(bedmachine['x'])))
assert(np.abs(np.max(np.diff(bedmachine['x']))) == bedmachine_spacing_m)
assert(np.abs(np.min(np.diff(bedmachine['x']))) == bedmachine_spacing_m)
assert(np.abs(np.max(np.diff(bedmachine['y']))) == bedmachine_spacing_m)
assert(np.abs(np.min(np.diff(bedmachine['y']))) == bedmachine_spacing_m)

In [ ]:
crs_3413 = ccrs.Stereographic(central_latitude=90, central_longitude=-45, true_scale_latitude=70)

def plot_on_map(da, **kwargs):
    if 'cmap' not in kwargs:
        if da.min() >= 0:
            kwargs['cmap'] = 'OrRd'
        else:
            kwargs['cmap'] = 'RdBu_r'
            if 'clim' not in kwargs:
                abs_max = float((np.abs(da)).max())
                kwargs['clim'] = (-abs_max, abs_max)

    plot = da.hvplot.image(x='x', y='y', geo=True, rasterize=True, crs=crs_3413, **kwargs)
    plot = plot * gv.feature.coastline()
    return plot

In [ ]:
smoothing_kernel_std = 5000 # m

us_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, surface_velocity['vx'], kwargs={'sigma': smoothing_kernel_std / surface_velocity_spacing_m})
vs_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, surface_velocity['vx'], kwargs={'sigma': smoothing_kernel_std / surface_velocity_spacing_m})
h_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, h, kwargs={'sigma': smoothing_kernel_std / surface_velocity_spacing_m})

In [ ]:
h_dx = h.differentiate('x')
h_dy = h.differentiate('y')
abs_grad_h = np.sqrt(h_dx**2 + h_dy**2)

rho=918.0
g=9.8
n=3
A0 = 3.985e-13 # s^-1 Pa^-3
Q = 60.0e3 # J mol^-1
R = 8.314 # J mol^-1 K^-1
T_rel_p = 273.15 - 20
A = A0 * np.exp(-Q/(R * T_rel_p)) * (365*60*60) # convert s^-1 to yr^-1

# def sia_u(x, y, z, rho=918.0, g=9.8, n=3, A=A):
#     us = us_filt.interp(x=x, y=y)
#     u_diff_from_surf = 2 * (rho * g)**n * abs_grad_h.interp(x=x, y=y)**(n-1) * h_dx.interp(x=x, y=y) * A * (h.interp(x=x,y=y) - z)**(n+1) / (n+1)
#     return us + u_diff_from_surf


In [ ]:
u_at_depth = us_filt + (2 * (rho * g)**n * abs_grad_h**(n-1) * h_dx * A * (h * 0.5)**(n+1) / (n+1))
v_at_depth = vs_filt + (2 * (rho * g)**n * abs_grad_h**(n-1) * h_dy * A * (h * 0.5)**(n+1) / (n+1))

In [ ]:
surf_vel_mag = np.sqrt(us_filt**2 + vs_filt**2)
vel_at_depth_mag = np.sqrt(u_at_depth**2 + v_at_depth**2)
plot_on_map(surf_vel_mag - vel_at_depth_mag, clim=(-1,1), height=500)

In [ ]:
plot_on_map(h_dx * 1000, height=500, clim=(-10, 10))

In [ ]:
plot_on_map(np.abs(us_filt) - np.abs(u_at_depth), height=500, clim=(-1, 1))

In [ ]:
plot_on_map(us_filt, height=500, clim=(-100, 100))

In [ ]:
us_filt.interp(x=-1.067e5, y=-2.267e6)

In [ ]:
[sia_u(-1.067e5, -2.267e6, z=z) for z in np.arange(0, 1000, 100)]

In [ ]:
A

In [ ]:
rho=918.0
g=9.8
n=3
x = -1.067e5
y = -2.267e6
#z = h.interp(x=x, y=y)
#2 * (rho * g)**n * abs_grad_h.interp(x=x, y=y)**(n-1) * h_dx.interp(x=x, y=y) * (A*60*60*365) * (h.interp(x=x,y=y) - z)**(n+1) / (n+1)

In [ ]:
plot_on_map(us_filt, height=500)

In [ ]:
def plot_on_map(da, **kwargs):
    if 'cmap' not in kwargs:
        if da.min() >= 0:
            kwargs['cmap'] = 'OrRd'
        else:
            kwargs['cmap'] = 'RdBu_r'
            if 'clim' not in kwargs:
                abs_max = float((np.abs(da)).max())
                kwargs['clim'] = (-abs_max, abs_max)

    plot = da.hvplot.image(x='x', y='y', geo=True, rasterize=True, crs=crs_3413, **kwargs)
    plot = plot * gv.feature.coastline()
    return plot

plot_on_map(h, height=500, title='Ice Thickness [m]')